### Full Pipeline Testing Notebook
Working on integrating the SSVEP stimulus with the EEG processing classes/modules

In [1]:
import sys
import os
sys.path.append("..")
import time
import numpy as np
from psychopy import visual, event, core, monitors
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from mvlearn.embed import CCA
from multiprocessing import Process, Queue

# Assuming the other modules (brainflow_stream, filtering, segmentation, classification) are available
from modules.brainflow_stream import *
from modules.filtering import *
from modules.segmentation import *
from modules.classification import *


pygame 2.5.2 (SDL 2.28.3, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# from modules.psychopy_ssvep_stim import *

from modules.multiprocessing_psychopy_ssvep_stim import *

In [6]:
serial_port = 'COM4' 
board_id = BoardIds.SYNTHETIC_BOARD # BoardIds.PLAYBACK_FILE_BOARD -> only if recorded with a support board (I believe...)
frequencies = [9.25, 11.25, 13.25, 15.25] # Stimulus frequencies; used for CCA & harmonic generation
buttons = ['Right', 'Left', 'Up', 'Down'] # Adds custom text to each box - must be same length as frequencies 
button_pos = [0, 2, 3, 1] # Assigns positions to custom text - must be same length as buttons
segment_duration = 4 # seconds
display = 0 # Which screen to display the stimulus paradigm on --> 0 is default
display_mode = "both"

# Static Variables - Probably don't need to touch :)
harmonics = np.arange(1, 4) # Generates the 1st, 2nd, & 3rd Harmonics
sampling_rate = BoardShim.get_sampling_rate(board_id)
n_samples = sampling_rate * segment_duration 


eeg_channels = BoardShim.get_eeg_names(board_id)
channel_names = ["O1", "O2", "Oz", "Pz", "P3", "P4", "POz", "P1"]
channel_mapping = dict(zip(eeg_channels, channel_names))

In [7]:
run_ssvep_stimulus_in_process(box_frequencies=frequencies, 
                              box_texts=buttons, 
                              box_text_indices=button_pos,
                              display_index=display, 
                              display_mode=display_mode)

<Process name='Process-1' pid=1776 parent=24196 started>

In [6]:

# Show board information
print(f"Sampling Rate: {sampling_rate}")
print(f"Default Channels: {eeg_channels}")
print(f"Channel Mapping: {channel_mapping}")

board = BrainFlowBoardSetup(board_id, serial_port)
# board.show_params() # Logger shows this info by default - this is another method to show
board.setup()

Sampling Rate: 250
Default Channels: ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'F5', 'F7', 'F3', 'F1', 'F2', 'F4', 'F6', 'F8']
Channel Mapping: {'Fz': 'O1', 'C3': 'O2', 'Cz': 'Oz', 'C4': 'Pz', 'Pz': 'P3', 'PO7': 'P4', 'Oz': 'POz', 'PO8': 'P1'}
Board setup and streaming started successfully


In [8]:
board.stop()


Streaming stopped
Session released


In [7]:
segmenter = PreProcess(board, segment_duration=segment_duration)
classifier = ClassifySSVEP(frequencies, harmonics, sampling_rate, n_samples, stack_harmonics=False)
classifier_stacked = ClassifySSVEP(frequencies, harmonics, sampling_rate, n_samples, stack_harmonics=True)

while True:
    segment = segmenter.get_segment()
    if segment is not None:
        
        # print(f"Segment shape: {segment.shape}")
        eeg_segment = segment[0:8, :]

        # Step 2: Filter the data
        filtered_segment = segmenter.filter_data(eeg_segment)
        print("Filtered data shape:", filtered_segment.shape)

        # Step 3: Use CCA to match the EEG & Reference (harmonic) signals
            # Unstacked Harmonics (testing)
        detected_freq, correlation = classifier.cca_analysis(filtered_segment)
            # Stacked Harmonics (testing)
        detected_freq_stacked, correlation_stacked = classifier_stacked.cca_analysis(filtered_segment)
        
        print(f"Detected frequency: {detected_freq} Hz with correlation: {correlation}")

        # Optionally save or process the data further
        # segmenter.save_data(filtered_data, "filtered_data.csv")
        # segmenter.save_data(features, "features.csv")

    # Sleep for a while to collect new data
    time.sleep(segmenter.segment_duration)

NameError: name 'PreProcess' is not defined